# Insert data from Jupyter notebook to Google Sheet

How to use SQL Server Views and Stored Procedures, when creating dataframes:
- View is a virtual table based on the result-set of an SQL statement. A view contains rows and columns, just like a real table. The fields in a view are fields from one or more real tables in the database. You can add SQL statements and functions to a view and present the data as if the data were coming from one single table.
- Stored procedure is a prepared SQL code that you can save, so the code can be reused over and over again. So if you have an SQL query that you write over and over again, save it as a stored procedure, and then just call it to execute it. You can also pass parameters to a stored procedure, so that the stored procedure can act based on the parameter value(s) that is passed.

NOTE: Google Sheet size is limited (40 000 rows, 18 278 columns, 200 sheets per workbook). Must use limited queries according to what you want to visualize.

- Create a Service Account, that is used to make authorized API calls to Google Cloud Services.
- Create dataframe by using SQL query.
- Connect Python to Google Sheets.
- Insert data to Google Sheet using df2gspread upload.

In [1]:
# Import libraries
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
from df2gspread import df2gspread as d2g
# Import libraries for SQL Server
import pyodbc
import sqlalchemy
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

#### Google Sheet connection

In [23]:
# Connect to your service account, API service account must create in Google Cloud
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('stately-block-356119-aaa54f3aa304.json', scope)
gc = gspread.authorize(credentials)

In [24]:
# Find the Google Sheet key of the spreadsheet you want to import, key is part of the url of the spreadsheet

# Remember to share: Go to your Google Sheet and click Share. Paste the client_email into people box and click Send.
# client_email can found from Service Account Credentials JSON file.

spreadsheet_key = 'Temporarily hidden'
wks_name = 'Onnettomuudet_2020'

#### SQL Server connection

In [25]:
# Establish connection to SQL Server
conn = 'DRIVER={ODBC Driver 17 for SQL Server};server=DESKTOP-Q88A49I\SQLEXPRESS;database=Onnettomuudet;trusted_connection=Yes;'
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn})
# Create engine between python and database
engine = create_engine(connection_url)

# Database parameters
Database = 'Onnettomuudet'

#### Query for SQL Server by using db Views or Stored Procedures

In [26]:
# Database Views
#sqlcommand_v1 = 'SELECT * FROM vuos_maak_onnett;'
#sqlcommand_v2 = 'SELECT * FROM vuos_maak_onnett_tyyp;'
#sqlcommand_v3 = 'SELECT * FROM vuos_vak_onnett_olos;'
#sqlcommand_v4 = 'SELECT * FROM vuos_vak_onnett_paikka;'
#sqlcommand_v5 = 'SELECT * FROM vuos_onnett_paikka;'
#sqlcommand_v6 = 'SELECT * FROM vuos_onnett_paikka_osall;'

# Database Procedures
vuosi = 2020
#sqlcommand_p1 = 'EXEC uspGetLoukk_Osall '+str(vuosi)
#sqlcommand_p2 = 'EXEC uspGetKuoll_Osall '+str(vuosi)
#sqlcommand_p3 = 'EXEC uspGetVakOnnett_Olos '+str(vuosi)
sqlcommand_p4 = 'EXEC uspGetVakOnnett_Paikka '+str(vuosi)

#### Create selected Query and insert results to dataframe

In [27]:
query = pd.read_sql_query(sqlcommand_p4,con=engine)
df = pd.DataFrame(query)

In [28]:
# Do some manipulation of the data
df

,Vuosi,Onnett_nro,Kuukausi,Viikonpäivä,Tunti,Vakavuus,Loukkaantuneet,Kuolleet,Osallisen_laji,Kuolleita,...,Valoisuus,Sää,Lämpötila,Maakunta,Maak_Loc,Väestö,Kunta,Katuosoite,position.lat,position.lon
0,2020,9621317,1,Sunnuntai,16.0,Loukkaantumiseen johtanut,1,0,henkilöauto,0,...,päivänvalo,kirkas,4.0,Varsinais-Suomi,FI-19,479341,Naantali,ARMONLAAKSONTIE X RUONAN YHDYSTIE,60.473389,22.052501
1,2020,9621317,1,Sunnuntai,16.0,Loukkaantumiseen johtanut,1,0,henkilöauto,0,...,päivänvalo,kirkas,4.0,Varsinais-Suomi,FI-19,479341,Naantali,ARMONLAAKSONTIE X RUONAN YHDYSTIE,60.473389,22.052501
2,2020,9621318,1,Tiistai,19.0,Loukkaantumiseen johtanut,1,0,henkilöauto,0,...,tie valaistu,vesisade,4.0,Pirkanmaa,FI-11,519872,Ikaalinen,JYLHÄNTIE,63.098905,23.084022
3,2020,9621318,1,Tiistai,19.0,Loukkaantumiseen johtanut,1,0,jalankulkija,0,...,tie valaistu,vesisade,4.0,Pirkanmaa,FI-11,519872,Ikaalinen,JYLHÄNTIE,63.098905,23.084022
4,2020,9621318,1,Tiistai,19.0,Loukkaantumiseen johtanut,1,0,jalankulkija,0,...,tie valaistu,vesisade,4.0,Pirkanmaa,FI-11,519872,Ikaalinen,JYLHÄNTIE,63.098905,23.084022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,2020,9945455,12,Perjantai,15.0,Loukkaantumiseen johtanut,1,0,henkilöauto,0,...,tie valaistu,pilvipouta,-8.0,Pohjanmaa,FI-12,175923,Kaskinen,TEOLLISUUSTIE 2,66.491480,25.688753
6257,2020,9955039,5,Sunnuntai,15.0,Kuolemaan johtanut,0,1,polkupyöräilijä,1,...,päivänvalo,Ei arvoa,19.0,Pirkanmaa,FI-11,519872,Ikaalinen,RAJA-AHONTIE,62.040650,24.719243
6258,2020,9955055,5,Keskiviikko,17.0,Kuolemaan johtanut,0,1,polkupyöräilijä,1,...,päivänvalo,Ei arvoa,8.0,Satakunta,FI-17,216752,Rauma,Lopentie 19,60.732131,24.763740
6259,2020,9955056,12,Torstai,22.0,Loukkaantumiseen johtanut,2,0,henkilöauto,0,...,tie valaistu,lumisade,-1.0,Pohjanmaa,FI-12,175923,Kaskinen,NEULAMÄENTIE,62.884217,27.622015


#### Insert data to Google Sheet

In [29]:
# Save the data to a new sheet in the dataframe
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Onnettomuudet_2020' id:754392154>

# Get Google sheet data to the Jupyter notebook

In [ ]:
# Import the data from your google spreadsheet into Jupyter

# Remember to share: Go to your Google Sheet and click Share. Paste the client_email into people box and click Send.
# client_email can found from Service Account Credentials JSON file.

book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Onnettomuudet")
table = worksheet.get_all_values()

In [ ]:
# Convert the sheet data into a pandas dataframe

# Convert table data into a dataframe
df = pd.DataFrame(table[1:], columns=table[0])
# Convert number strings to floats and ints
df = df.apply(pd.to_numeric, errors='ignore')
# Convert date strings to datetime format
df['Aikaleima'] = pd.to_datetime(df['Aikaleima'], format='%d.%m.%Y klo %H.%M.%S')
#df.head()